<a href="https://colab.research.google.com/github/merliou/term_paper_css/blob/main/bert_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Installiere die notwendigen Bibliotheken
!pip install bertopic
!pip install umap-learn
!pip install pandas
!pip install numpy

# Importiere die notwendigen Bibliotheken
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
#from hdbscan import HDBSCAN

# Lade die CSV-Datei
data = pd.read_csv("10_data_covs_avg_no_doubles.csv", header=0)

## draw 10 percent random sample
#np.random.seed(161)
#data = data.sample(frac=0.1)

data['ort'] = data['ort']
data['bundesland'] = data['bundesland']
data['rating_rez'] = data['bewertung']
data['rezension'] = data['rezension']
data['traeger'] = data['Traeger']
data['typ'] = data['EinrichtungsTyp']
data['versorgungsstufe'] = data['Allgemeine_Notfallversorgung']
data['bettenzahl'] = data['INSG']
data['rating_avg'] = data['rating']
data['number_of_ratings'] = data['total_ratings']

data['bundesland'].value_counts()
data['rating_rez'].value_counts()
data['traeger'].value_counts()
data['typ'].value_counts()
data['versorgungsstufe'].value_counts()
data['rating_avg'].value_counts()


# Zeige die ersten 3 Zeilen der Daten an
print(data.iloc[0:3])

## Bereite die Daten vor (falls nötig)
## In diesem Fall verwenden wir die Spalte 'rezension' für die Textanalyse
#documents = data['rezension'].tolist()
#
## Erstelle das UMAP-Modell
#umap_model = UMAP(random_state=1337)
#
## Erstelle das BERTopic-Modell
#topic_model = BERTopic(umap_model=umap_model)
#
## Führe das Modell auf den Dokumenten aus
#topics, probs = topic_model.fit_transform(documents)
#
## Zeige die häufigsten Themen an
#topic_info = topic_model.get_topic_info()
#print(topic_info)

,count
bundesland,
5.0,2347
9.0,2065
8.0,1432
3.0,1002
6.0,994
7.0,582
1.0,570
11.0,493
14.0,458


Data cleaning

In [53]:
# Überprüfe auf fehlende Werte in der Spalte 'rezension'
print(data['rezension'].isnull().sum())

# Entferne Zeilen mit fehlenden Werten
data = data.dropna(subset=['rezension'])

## Oder ersetze fehlende Werte durch einen leeren String
#data['rezension'] = data['rezension'].fillna("")

# Extrahiere die Dokumente
documents = data['rezension'].tolist()

# Überprüfe die ersten 5 Dokumente
print(documents[:5])

# Überprüfe den Datentyp der Spalte 'rezension'
print(data['rezension'].dtype)

data['rezension'] = data['rezension'].astype(str)

Modell ausführen

In [57]:
## Extrahiere die Dokumente
#documents = data['rezension'].tolist()

# Erstelle das BERTopic-Modell
topic_model = BERTopic(min_topic_size=9) # Minimalanzahl an Topics einstellen

# Führe das Modell auf den Dokumenten aus
topics, probs = topic_model.fit_transform(documents)

# Zeige die Themeninformationen an
topic_info = topic_model.get_topic_info()
print(topic_info)

    Topic  Count                                 Name  \
0      -1   6319                   -1_und_ich_die_der   
1       0    648                  0_klinik_die_und_in   
2       1    397                    1_die_und_der_ich   
3       2    394            2_krankenhaus_das_und_ist   
4       3    337           3_krankenhaus_ich_nicht_in   
..    ...    ...                                  ...   
76     75     10  75_notaufnahme_in_freitagabend_eine   
77     76     10            76_geholfen_mir_bar_super   
78     77     10           77_zimmer_wurde_fehler_ich   
79     78      9    78_personal_alternativen_ist_oder   
80     79      9   79_mhh_ihrem_patienten_krankenhaus   

                                       Representation  \
0   [und, ich, die, der, das, war, sehr, in, nicht...   
1   [klinik, die, und, in, der, ich, ist, zu, das,...   
2   [die, und, der, ich, nicht, zu, in, mit, dass,...   
3   [krankenhaus, das, und, ist, sehr, ich, ein, d...   
4   [krankenhaus, ich, nicht, 

Modell 2: Dok's in einzelne Sätze gesplittet

In [91]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

def split_into_sentences(text):
  sentences = nltk.sent_tokenize(text, language='german') # use language='german' for German text.
  return sentences

# Wende die Funktion auf alle Rezensionen an
split_documents = [split_into_sentences(doc) for doc in documents]
# flatten the list for Bertopic
all_sentences = [sentence for sublist in split_documents for sentence in sublist]

# Erstelle und trainiere das BERTopic-Modell mit den Sätzen
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(all_sentences)

In [60]:
# Zeige die Dokumentinformationen an
document_info = topic_model.get_document_info(documents)
print(document_info)

# Visualisiere die Ergebnisse
# Themen-Distanz
topic_model.visualize_topics()

In [62]:
# Wort-Scores
topic_model.visualize_barchart() # erste 10 topics

In [63]:
# Themen-Similarität
topic_model.visualize_heatmap()

In [96]:
# Themen-Hierarchie
topic_model.visualize_hierarchy()

pip install -U kaleido
import kaleido
topic_model.visualize_hierarchy().write_image("hierarchical_clustering.png") # als Bild speichern



ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
# Zeige spezifische Themen an (z.B. Thema 49)
specific_topic = topic_model.get_topic(49)
print(specific_topic)

# Zeige repräsentative Dokumente für spezifische Themen an (z.B. Thema 49 und 82)
representative_docs_49 = topic_model.get_representative_docs(topic=49)
representative_docs_82 = topic_model.get_representative_docs(topic=82)
print(representative_docs_49)
print(representative_docs_82)

[('station', np.float64(0.03075882698078743)), ('klinik', np.float64(0.027546367922963364)), ('bis', np.float64(0.0220624117142804)), ('31', np.float64(0.019609307870642197)), ('sehr', np.float64(0.019021210113480475)), ('vom', np.float64(0.017089858739553477)), ('mich', np.float64(0.015538967779543769)), ('an', np.float64(0.015220149515462798)), ('stationr', np.float64(0.014620596392466096)), ('ihr', np.float64(0.013943521142597984))]
['Eine sehr gute Klinik. Ich war 1 Woche stationär auf St. 31 und wurde sehr gut betreut. Ein großes Lob und Dankeschön an Dr. M. Stockbauer und OÄ Dr. Wallisch für die hervorragende Behandlung.\nAlle Mitarbeiter/innen auf St. 31 geben ihr bestes und waren überaus freundlich und bemüht.\nDer einzigste Minuspunkt ist das Essen, leider nicht sehr gut, aber auch das überlebt man :-)\nViele Grüße,  Claudia Grimm', 'Ich war vor einer Woche in der Klinik. Angefangen bei der Patientenaufnahme über das OP-Team bis zur Station sehr freundliche und hilfsbereite Me

Fine tuning

In [81]:
# Benutzerdefinierte Themen-Labels setzen
topic_model.set_topic_labels({49: "Krankenhaus Erfahrungen", 82: "Psychiatrische Behandlung"})
print(topic_model.get_topic_info(49))
print(topic_model.get_topic_info(82))

# Themen pro Klasse (falls eine Klassenspalte vorhanden ist)
classes = data['versorgungsstufe'] # bei versorgungsstufe gibts Probleme

topics_per_class = topic_model.topics_per_class(documents, classes=classes)
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=31)

# Ändere das Clustering-Modell zu KMeans
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=30)  # Verwende KMeans-Clustering mit ... Themen
topic_model_kmeans = BERTopic(hdbscan_model=cluster_model)
topicsKmeans, probsKmeans = topic_model_kmeans.fit_transform(documents)

# Zeige die Themeninformationen für das KMeans-Modell an
topic_info_kmeans = topic_model_kmeans.get_topic_info()
print(topic_info_kmeans)

# Output in Tabellenform
html_table = topic_info.to_html()
with open("topic_info.html", "w", encoding='utf-8') as f:
    f.write(html_table)

# Zeige die Dokumentinformationen für das KMeans-Modell an
document_info_kmeans = topic_model_kmeans.get_document_info(documents)
print(document_info_kmeans)

## Geführte Themenmodelle (Guided Topic Models)
#seed_topic_list = [["warte"],
#                   ["schmerz"],
#                   ["frechheit"],
#                   ["freundlich"]]

topic_model_guided = BERTopic(seed_topic_list=seed_topic_list)
topics_guided, probs_guided = topic_model_guided.fit_transform(documents)

# Zeige die Themeninformationen für das geführte Modell an
topic_info_guided = topic_model_guided.get_topic_info()
print(topic_info_guided)

# Visualisiere die Ergebnisse des geführten Modells
topic_model_guided.visualize_barchart(n_words=8, top_n_topics=12)

# Zeige die Dokumentinformationen für das geführte Modell an
document_info_guided = topic_model_guided.get_document_info(documents)
print(document_info_guided)